In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df1 = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
len(df.select_dtypes(include='object').columns)

In [ ]:
df.head()

In [ ]:
df.shape,df1.shape

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
na = df.isna().sum()/len(df)

In [ ]:
na[na>.50]

In [ ]:
df = df.drop(columns=['PoolQC','Id','Alley','Fence','MiscFeature'])
df1 = df1.drop(columns=['PoolQC','Id','Alley','Fence','MiscFeature'])

In [ ]:
obj = df.select_dtypes(include='object').columns

In [ ]:
plt.figure(figsize=(25,20))
sns.heatmap(df.corr(),annot=True)

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [ ]:
for i in obj:
    df[i] = df[i].fillna('aaaaaaaa')
    df[i] = df[i].astype(str)

In [ ]:
for i in obj:
    df1[i] = df1[i].fillna('aaaaaaaa')
    df1[i] = df1[i].astype(str)

In [ ]:
for i in obj:
    df[i]= label_encoder.fit_transform(df[i])
    df1[i]= label_encoder.fit_transform(df1[i])

In [ ]:
obj

In [ ]:
for i in df.select_dtypes(include='object').columns:
    print('____________________________________________________')
    print(i,df[i].nunique())
    

In [ ]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')

In [ ]:
from sklearn.experimental import enable_iterative_imputer  # noqa

from sklearn.impute import IterativeImputer

In [ ]:
imp = IterativeImputer(random_state=0)

In [ ]:
df = pd.DataFrame(imp.fit_transform(df),columns=list(df.columns))
df1 = pd.DataFrame(imp.fit_transform(df1),columns=list(df1.columns))

In [ ]:
df

In [ ]:
model = CatBoostRegressor(iterations=4000,verbose=False)
#model = XGBRegressor(n_estimators=50)

In [ ]:
X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
model.fit(X_train,y_train)
#model.fit(X_train,y_train,verbose=False)
model.score(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_log_error
np.sqrt(mean_squared_log_error(y_test, y_pred))

In [ ]:
model.fit(X,y,verbose=False)
model.score(X,y)

In [ ]:
y_pred = model.predict(df1)

In [ ]:
sub = pd.DataFrame({'id':range(1461,1461+len(df1)),'SalePrice':y_pred})
sub.to_csv('submission.csv',index=False)